# Big project activity

## Optimal charging station location

### 1.Introduction

Consider a long linear cycle path as Vento, or the Danube cycle path. The cycle path usually runs along the banks of a river with scarce tourist interest. However, from the main course of the cycle path, it is possible to reach places of tourist interest in making small detours.  

The rapid growth of e-bike ridership is proposing the problem of deploying a suitable charging infrastructure. The charging stations should be placed in strategic positions so as to guarantee a coverage of the whole cycle path. However, since the charging operations require a non-negligible time, the charging station should be positioned in places where alternative activities could be carried out, as restaurants, museums, swimming pool, or other amenities. Moreover, the presence of a charging station could also induce e-cyclists to discover new places and generate positive externalities.



### 2.Decision problem
We can represent the cycle path as a graph where the set of nodes $H = \{1,\ldots, n\}$ corresponds to the tourist sites that may host a charging station.
In addition, we are given the distances between touristic sites ($d_{ij},$ with $ i,j =1,\ldots,n$). Let $c_i$ be the cost of installing a charging station in site $i, i=1\ldots, n$.


The problem is, given a budget $b$, determine the subset of sites $S\subseteq H$ where to install the charging stations so that the total cost is not higher than $b$ and the maximum distance between consecutive charging stations is minimized.
Consider that the cyclist has to visit all the touristic destinations in a consecutive way.



### 3.Problem characteristics
There are 2 csv files that contain the information of the cycle way, they are essential to build the equivalent graph:


*    in the "nodes.csv" file, there are all the destinations that the cyclist can reach, with their spatial coordinates and the value of installation costs related to that destination. Consider that the "tourist-dest-id" is not the graph node number, but it is a unique id to identify the destination.
*   in the "OD.csv" you can find all the arcs between two different nodes, keep attention that the condition of visiting consecutive touristic destination must be respected.

The set of nodes $N$ is defined by $\{0,1,\ldots,n, n+1\}$.  The Arcs $A$ correspond to the portion of cycle path between two consecutive charging stations. We assume that potentially e-riders will visit all sites along the way, making the suitable deviations and going back to the main path at the initial point of the detour.
The cost associated with each arc $(i,j)$ is given by $c_j$, thus the cost of installing a charging station in $j$. These costs are defined for all arcs in $A$, while they are set to 0 for all the arcs that arrive in the last node.
The path starts in node $s = 0$ and ends in node $t = n+1$, these two nodes are connected to the nearest touristic site with an arc of null length.

### 4.Example of a linear path with deviation
![picture](https://drive.google.com/uc?export=view&id=1w16bHtbu0FGGL-UntxeqxD7244D3eHbJ)

### 5.Requirements
The requirements of the problem are:


*   the maximum running time of the algorithm must be 5 minutes, so set the proper timer
*   create the equivalent graph and display it on a xy-plot
*   find the solution for the basic scenario, with a mip model, displaying the solution with a xy-plot, the budget constraint is $b = 10000\ € $.
*   Find the optimal solution for 5 different values of budget in the range $[10000, 100000]$. Select the values of the budget so as to have different charger locations.

  You have to motivate your choice and the solution you get. They can also be not common solution if they are well motivated. To support your decision and explanations, you can print plots or tables. You can also compare different scenarios.


   
If you have some doubts related to the parametric analysis, prof. Cubillos uploaded a notebook with the solution on WeBeeP and you can have a look there.

### Insert student name and student ID

student1:

ID1:

student2:

ID2:

student3

ID3:



In [1]:
#install libraries
!pip install mip
!pip install --upgrade cffi==1.15.0

In [2]:
#import libraries
import importlib
import cffi

importlib.reload(cffi)
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd
import time
import random

/tmp/ipykernel_228814/1401400862.py:9: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
# set the budget
budget: int = 10_000
# budget = random.randint(10000, 100000)
print(f"Budget: {budget:,.2f} €")

Budget: 10,000.00 €


In [4]:
# Load nodes data from CSV
nodes_data: pd.DataFrame = pd.read_csv('nodes.csv')

# Display information about candidate touristic sites and nodes
num_nodes: int = len(nodes_data) + 2
print(f"Number of candidate touristic sites: {len(nodes_data)}")
print(f"Number of Nodes (including source and sink): {num_nodes}")

# Load OD data from CSV and map node IDs
od_data: pd.DataFrame = pd.read_csv('OD.csv')
node_id_mapping: dict[int, int] = {node_id: i for i, node_id in enumerate(nodes_data['tourist_dest_id'])}
od_data[['origin_id', 'destination_id']] = od_data[['origin_id', 'destination_id']].apply(
    lambda x: x.map(node_id_mapping))

# Map node IDs in nodes_data
nodes_data['tourist_dest_id'] = nodes_data['tourist_dest_id'].map(node_id_mapping)

# Display the number of arcs and the node ID mapping
print(f"Number of arcs: {len(od_data)}")
print(f"Node ID Mapping: {node_id_mapping}")


Number of candidate touristic sites: 44
Number of Nodes (including source and sink): 46
Number of arcs: 1936
Node ID Mapping: {0: 0, 17: 1, 18: 2, 20: 3, 21: 4, 23: 5, 24: 6, 25: 7, 29: 8, 30: 9, 31: 10, 32: 11, 33: 12, 35: 13, 36: 14, 38: 15, 39: 16, 40: 17, 41: 18, 48: 19, 52: 20, 53: 21, 54: 22, 57: 23, 58: 24, 60: 25, 61: 26, 62: 27, 63: 28, 64: 29, 66: 30, 67: 31, 68: 32, 69: 33, 77: 34, 82: 35, 83: 36, 84: 37, 85: 38, 86: 39, 87: 40, 88: 41, 89: 42, 90: 43}


In [5]:
import folium
from branca.element import Figure


def create_folium_map(candidate_sites_coordinates: pd.DataFrame, center: list[float]) -> folium.Map:
    """
    Creates a Folium map with CircleMarkers for candidate sites.

    Parameters:
    - candidate_sites_coordinates (pd.DataFrame): DataFrame containing candidate sites data with 'y (latitude)', 'x (longitude)',
      'Comune', 'Piazza', and 'tourist_dest_id'.
    - center (List[float]): List representing the center location of the map [latitude, longitude].

    Returns:
    - folium.Map: Folium map with CircleMarkers for candidate sites.
    """
    # Initialize the map
    map = folium.Map(location=center, zoom_start=10)

    # Add CircleMarkers for each row in candidate_sites_coordinates
    for _, row in candidate_sites_coordinates.iterrows():
        popup_text = f"{row['Comune']} - {row['Piazza']}" if pd.notna(row['Piazza']) else f"{row['Comune']}"

        folium.CircleMarker(
            location=[row['y (latitude)'], row['x (longitude)']],
            radius=5,
            color='blue',
            fill=True,
            fill_color='blue',
            fill_opacity=0.7,
            popup=popup_text,
            tooltip=str(row['tourist_dest_id'])
        ).add_to(map)

    return map


# Center location
map_center: list[float] = [44.92803444, 10.52108953]

# Create Folium map with the specified center
nodes_map: folium.Map = create_folium_map(candidate_sites_coordinates=nodes_data, center=map_center)

# Create a Figure and add the Folium map to it
fig: Figure = Figure(width=1000, height=700)
fig.add_child(nodes_map)

# Display the Figure
fig

In [6]:
def find_first_node(nodes_data: pd.DataFrame) -> int:
    """
    Finds the first node based on the lowest value of longitude.

    Parameters:
    - nodes_data (pd.DataFrame): DataFrame containing node data with 'tourist_dest_id', 'x (longitude)', 'y (latitude)', etc.

    Returns:
    - int: The tourist destination ID of the node with the lowest longitude.
    """
    min_longitude_row = nodes_data.loc[nodes_data['x (longitude)'].idxmin()]
    return min_longitude_row['tourist_dest_id']


def find_last_node(nodes_data: pd.DataFrame) -> int:
    """
    Finds the last node based on the highest value of longitude.

    Parameters:
    - nodes_data (pd.DataFrame): DataFrame containing node data with 'tourist_dest_id', 'x (longitude)', 'y (latitude)', etc.

    Returns:
    - int: The tourist destination ID of the node with the highest longitude.
    """
    max_longitude_row = nodes_data.loc[nodes_data['x (longitude)'].idxmax()]
    return max_longitude_row['tourist_dest_id']


In [7]:
#set the timer
# Starting time
start_time = time.time()
#TO DO

## Shortest path -> Graph
This algorithm is used to find the shortest path between two nodes (head, tail). In this case:
- Head: 0
- Tail: 20 (52 in the original OD file) -> selected because is the furthest from 0 if we look at the path (this selection of the node could be automatized)

In [ ]:
import mip
import math

n = len(od_data)
distance_matrix = [[0 for j in range(n)] for i in range(n)]

for _, row in od_data.iterrows():
    i, j = int(row['origin_id']), int(row['destination_id'])
    if math.isnan(row['distance [m]']):
        distance_matrix[i][j] = 0
    else:
        distance_matrix[i][j] = row['distance [m]']

# Create model
m = mip.Model()

# define the variables
x = [[m.add_var(var_type=mip.BINARY) for j in range(n)] for i in range(n)]
print(f"Number y (arc ij is selected or not) of variables: {len(x)}")

#  define the constraints
first_node = find_first_node(nodes_data)
last_node = find_last_node(nodes_data)

for i in range(n):
    m += x[i][i] == 0
    if i == last_node:
        m += mip.xsum(x[i][j] for j in range(n) if i != j) == 0  # set the sink node to 20 -> no outgoing arcs
    else:
        m += mip.xsum(x[i][j] for j in range(n) if i != j) == 1

    if i == first_node:
        m += mip.xsum(x[j][i] for j in range(n) if i != j) == 0  # set the source node to 0 -> no incoming arcs
    else:
        m += mip.xsum(x[j][i] for j in range(n) if i != j) == 1

    # Sub-tour elimination
u = [m.add_var() for i in range(n)]
for i in range(1, n):
    for j in range(1, n):
        if i != j:
            m += u[i] - u[j] + (n - 1) * x[i][j] <= n - 2

# optimize objective function
m.objective = mip.minimize(mip.xsum(distance_matrix[i][j] * x[i][j] for i in range(n) for j in range(n)))

m.optimize()

path = []
for i in range(n):
    for j in range(n):
        if x[i][j].x > 0.5:  # If the path is chosen in the solution
            path.append((i, j))

# Display the solution path
print("Optimal path:", path)

current_node = 0

# Initialize the ordered path list with the start node
ordered_path = [current_node]

# Loop until we reach the end node (20 in this case)
while current_node != 20:
    for j in range(n):
        if x[current_node][j].x >= 0.99:  # If the path goes from current_node to j
            ordered_path.append(j)  # Add j to the path
            current_node = j  # Update the current node to j
            break

# Print the ordered path
print("Ordered path from 0 to 20:", ordered_path)

total_distance = 0

# Iterate over the path to sum the distances
for i in range(len(ordered_path) - 1):
    total_distance += distance_matrix[ordered_path[i]][ordered_path[i + 1]]

print("Total distance from start to finish:", total_distance)

Number y (arc ij is selected or not) of variables: 1936
Welcome to the CBC MILP Solver 
Version: Trunk
Build Date: Oct 24 2021 

Starting solution of the Linear programming relaxation problem using Dual Simplex
Coin0506I Presolve 3746160 (-1938) rows, 3744226 (-5806) columns and 18709518 (-11608) elements
Clp0014I Perturbing problem by 0.001% of 252710.17 - largest nonzero change 0.45994189 ( 0.26471931%) - largest zero change 0.25281015
Clp0006I 2000  Obj 2584.5732 Primal inf 5441.6884 (3164)
Clp0006I 4000  Obj 2604.1834 Primal inf 8133.7238 (2141)
Clp0006I 5000  Obj 2604.1916 Primal inf 63.834929 (1166)
Clp0006I 6000  Obj 2604.1867 Primal inf 22.853597 (510)
Clp0006I 6426  Obj 2604.1856
Clp0000I Optimal - objective value 0
Coin0511I After Postsolve, objective 0, infeasibilities - dual 0 (0), primal 0 (0)
Clp0032I Optimal objective 0 - 6426 iterations time 72.282, Presolve 7.77

Starting MIP optimization


In [ ]:
G = nx.DiGraph()

# Add nodes
for i, row in nodes_data.iterrows():
    G.add_node(row["tourist_dest_id"], pos=(row["x (longitude)"], row["y (latitude)"]),
               label=row["Comune"] + " - " + str(row["Piazza"]))

# Add edges for the optimal path
for i in range(len(path)):
    origin, destination = path[i]
    G.add_edge(origin, destination, distance=distance_matrix[origin][destination])

# Draw the graph
pos = nx.get_node_attributes(G, "pos")
nx.draw(G, pos, with_labels=True, node_size=500, font_size=8, font_color="black", font_weight="bold",
        node_color="skyblue", edge_color="gray", arrowsize=10)

plt.show()

If you want to reset the graph to its original state, run the cell on top of this one again.

In [ ]:
max_distance = 5_0000
count = 0

for i, node in enumerate(ordered_path):
    for j in range(n):
        # Check if the node is not the next node in the path and is within the distance limit
        if j != node and (i == len(ordered_path) - 1 or j != ordered_path[i + 1]) and distance_matrix[node][
            j] <= max_distance:
            # Add an edge only if there isn't already an edge in either direction
            if not G.has_edge(node, j) and not G.has_edge(j, node):
                count = count + 1
                G.add_edge(node, j, distance=distance_matrix[node][j])

# Update the graph
pos = nx.get_node_attributes(G, "pos")
plt.figure(figsize=(15, 15))
nx.draw(G, pos, with_labels=True, node_size=500, font_size=8, font_color="black", font_weight="bold",
        node_color="skyblue", edge_color="gray", arrowsize=10)

plt.show()

In [ ]:
import mip

# Create model
m = mip.Model()

# define the variables

# TO DO


#  define the constraints
# Budget Constraint

# Flow Conservation Constraints


# TO DO


# optimize objective function
# TO DO


m.optimize()

In [ ]:
print(m.objective_value)

In [ ]:
#plot the graph

#TO DO

In [ ]:
# parametric analysis


#TO DO



